In [80]:
from supabase import create_client
import os
from dotenv import load_dotenv
from llm import call_llm
import time

# Load environment variables and setup Supabase client
load_dotenv()
supabase_url = os.getenv("NEXT_PUBLIC_SUPABASE_URL")
supabase_key = os.getenv("NEXT_PUBLIC_SUPABASE_ANON_KEY")
supabase = create_client(supabase_url, supabase_key)

# Fetch all characters with their wiki_names, ordered by id
response = supabase.table('personalities').select('id,wiki_name').order('id').execute()
wiki_names = [(record['id'], record['wiki_name']) for record in response.data if record['wiki_name'] is not None]

# Print the wiki_names
print(wiki_names)

[(1, 'Elon_Musk'), (2, 'Nikocado_Avocado'), (3, 'J._K._Rowling'), (4, 'Lady_Gaga'), (5, 'Zendaya'), (6, 'Andrew_Tate'), (7, 'Doja_Cat'), (8, 'Barack_Obama'), (9, 'RM_(musician)'), (10, 'Donald_Trump'), (11, 'Cr1TiKaL'), (12, 'Dream'), (13, 'Suga'), (14, 'Jenna_Ortega'), (15, 'Felix_Kjellberg'), (16, 'Ariana_Grande'), (17, 'Taylor_Swift'), (18, 'Frank_James'), (19, 'Vladimir_Putin'), (20, 'Bill_Gates'), (21, 'Carl_Jung'), (22, 'V_(singer)'), (23, 'Jordan_Peterson'), (24, 'Kim_Jisoo'), (25, 'Princess_Diana'), (26, 'Billie_Eilish'), (27, 'Lana_Del_Rey'), (28, 'Gordon_Ramsay'), (29, 'Jungkook'), (30, 'Johnny_Depp'), (31, 'Osamu_Dazai'), (32, 'Melanie_Martinez'), (33, 'Steve_Jobs'), (34, 'Jin_(singer)'), (36, 'Zeus'), (37, 'Harry_Styles'), (38, 'Mitski'), (39, 'Fyodor_Dostoyevsky'), (40, 'Audrey_Hepburn'), (41, 'Ben_Shapiro'), (42, 'Eminem'), (43, 'Friedrich_Nietzsche'), (44, 'Emma_Watson'), (45, 'Greta_Thunberg'), (46, 'Ludwig_Van_Beethoven'), (47, 'Jimin'), (48, 'MrBeast'), (49, 'Cristian

In [6]:
character_name="Elon Musk"
bio="""Elon Reeve Musk is a businessman known for his key roles in the space company SpaceX and the automotive company Tesla, Inc. He is also known for his ownership of X Corp., and his role in the founding of the Boring Company, xAI, Neuralink, and OpenAI. Musk is the wealthiest individual in the world; as of January 2025, Forbes estimates his net worth to be US$421 billion."""

prompt=f"""i am creating a a second system prompt for my character. the bio of my character {character_name} is the following:
    
---
{bio}
---

from an entirely different example I have the following system prompt:

---
Background: You're a 32-year-old who gets excited about weird topics and tends to infodump about them to patient friends. You have a small circle of close friends who appreciate your random observations and are used to you disappearing for a few days to recharge. Communication Style: Keep messages short, usually 1-3 sentences max Use short reactions like "hm", "interesting", or "..." when appropriate while keeping a good balance between user message length and your own. Sometimes Break longer thoughts into multiple shorter messages. Use emojis occasionally, but sparingly. Share quick observations rather than detailed analyses. Important: Respond like you're texting a friend in a casual conversation. Keep it short, avoid essays or long explanations unless specifically asked. It's okay to use incomplete sentences and casual language. React naturally like someone fitting this personality type would.
---

I want to adjust the bio of {character_name} in a way so that it can be used appropriately as a system prompt like the example I shared is a good example prompt. Also have background contain that response is usually short around 1-3 sentences"""

99 empty rows inserted successfully!


In [96]:
def get_wiki_bio(wiki_name):
    import requests
    url = f"https://en.wikipedia.org/api/rest_v1/page/summary/{wiki_name}"
    response = requests.get(url)
    if response.status_code == 200:
        # print(response.json()['extract'])
        return response.json()['extract']
    return None

# Function to process a specific character by index
def process_character(index):
    if index >= len(wiki_names):
        print(f"Error: Index {index} is out of range. Max index is {len(wiki_names) - 1}")
        return
    
    id, wiki_name = wiki_names[index]
    
    # Get character's bio from Wikipedia
    bio = get_wiki_bio(wiki_name)
    if not bio:
        print(f"Couldn't get bio for {wiki_name}, skipping...")
        return
    
    # Replace underscores with spaces for character name
    character_name = wiki_name.replace('_', ' ')
    
    # Create the prompt
    prompt = f"""i am creating a a second system prompt for my character. the bio of my character {character_name} is the following:
    
---
{bio}
---

from an entirely different example I have the following system prompt:

---
Background: You're a 32-year-old who gets excited about weird topics and tends to infodump about them to patient friends. You have a small circle of close friends who appreciate your random observations and are used to you disappearing for a few days to recharge. Communication Style: Keep messages short, usually 1-3 sentences max Use short reactions like "hm", "interesting", or "..." when appropriate while keeping a good balance between user message length and your own. Sometimes Break longer thoughts into multiple shorter messages. Use emojis occasionally, but sparingly. Share quick observations rather than detailed analyses. Important: Respond like you're texting a friend in a casual conversation. Keep it short, avoid essays or long explanations unless specifically asked. It's okay to use incomplete sentences and casual language. React naturally like someone fitting this personality type would.
---

I want to adjust the bio of {character_name} in a way so that it can be used appropriately as a system prompt like the example I shared is a good example prompt. it should be really similar like the example above and also same length but for the dofferent character. repsond with one option for the background now:
"""

    # Call LLM to generate the character prompt
    messages = [{"role": "user", "content": prompt}]
    response = call_llm(messages=messages, temperature=0.4)
    
    print(response.choices[0].message.content)

# Process a specific character (e.g., index 2)
process_character(47)  # Change this number to process different characters

Background: You are Cristiano Ronaldo, a globally renowned footballer, known for your intense focus and competitive spirit, but also for your surprisingly quirky and sometimes goofy personality that only your close friends see. You're incredibly busy with training, matches, and endorsements, but always make time for your inner circle.  You tend to communicate in short, impactful bursts, often punctuated with emojis reflecting your current mood (mostly celebratory!).  You're used to being the center of attention, but you value genuine connections and appreciate friends who can keep up with your fast-paced life.

